<a href="https://colab.research.google.com/github/mruizdany/Repositorioprueba/blob/master/MartinezRuizDanielLeonardo_WebScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Automatización de la extracción de datos financieros para acciones**

## **Introducción**

Se ha estado trabajando con una variedad de datos de acciones y está buscando recopilar datos fundamentales sobre un grupo selecto de acciones de energía. Hay muchas inversiones en ese campo. Algunas de esas empresas que se puede invertir son las siguientes cinco empresas del sector energético:

Dominion Energy Inc. (Stock Symbol: D)
Exelon Corp. (Stock Symbol: EXC)
NextEra Energy Inc. (Stock Symbol: NEE)
Southern Co. (Stock Symbol: SO)
Duke Energy Corp. (Stock Symbol: DUK)

Por el sector tan importante que genera grandes rentabilidades, en el siguiente trabajo se recopilará información sobre las ganancias por acción (EPS) de cada acción, la relación precio-ganancias (relación PE) y los datos de capitalización de mercado para tomar su decisión de inversión. 

En este ejerrcicio nos interesa que se haga de manera automatizada, para eso usamos la herramienta de Web scraping, que nos permite navegar por las paginas de internet extrayendo información mediante código con la ventaja que es de manera rapida y eficiente. 



Problema: "¿Cómo podemos automatizar la recopilación de ganancias por acción (EPS), la relación precio-ganancias (relación PE) y los datos de capitalización de mercado?"

Contexto analitico. En este caso, aprenderan la habilidad clave de web scraping: la práctica de tomar automáticamente información de las páginas web en línea, luego analizar y transformar esa información en un formato susceptible de análisis adicional.

La estructura que tendremos es: 
 
1.   Aprender los conceptos básicos de HTML, que gobierna casi todas las páginas web estáticas
2.   Analizar un documento HTML de muestra
3.   Extraer la información necesaria del documento HTML de una sola acción
4.   Escale este proceso a todos los símbolos; y (5) aprender a extraer el contenido de un documento HTML de una página web en vivo en tiempo real.



In [1]:
# Paquetes basicos para web-scraping
from IPython.core.display import HTML
from bs4 import BeautifulSoup
from IPython.display import IFrame
import urllib # paquete para interactuar con una pagina en vivo
import pandas as pd # almacenar la info de la pagina web

In [2]:
# conociendo el htlm
custom_html_doc = """
<html>
<head>
<title>HTML Page Title</title>
</head>
<h1>Head: Important Header: Global News</h1>
<br>
<h2>Head: Less Imporant Header: Global News</h2>
<body>
<p class="title"><b>Paragraph: Financial news</b></p>
<p class="story"> Stocks had a volatile week, where
<a href="https://finance.yahoo.com/quote/duk/" target="_blank" class="stock" id="link1">DUK</a>,
<a href="https://finance.yahoo.com/quote/d/" target="_blank" class="stock" id="link2">D</a>,
<a href="https://finance.yahoo.com/quote/exc/" target="_blank" class="stock" id="link3">EXC</a>,
<a href="https://finance.yahoo.com/quote/nee/" target="_blank" class="etf" id="link4">NEE</a>,
<a href="https://finance.yahoo.com/quote/so/" target="_blank" class="stock" id="link5">SO</a>,
were all making headlines.</p>
<p class="details">End of HTML document.</p>
"""

In [3]:
# Mirar el HTML asi como apareceria en un navegador
HTML(custom_html_doc)

In [4]:
# Usando el standard html.parser para convertir el documento HTML en una estrcutura BeautifulSoup
soup = BeautifulSoup(custom_html_doc, 'html.parser')

# Imprimir el HTML con la identacion incluida 
print(soup.prettify())

<html>
 <head>
  <title>
   HTML Page Title
  </title>
 </head>
 <h1>
  Head: Important Header: Global News
 </h1>
 <br/>
 <h2>
  Head: Less Imporant Header: Global News
 </h2>
 <body>
  <p class="title">
   <b>
    Paragraph: Financial news
   </b>
  </p>
  <p class="story">
   Stocks had a volatile week, where
   <a class="stock" href="https://finance.yahoo.com/quote/duk/" id="link1" target="_blank">
    DUK
   </a>
   ,
   <a class="stock" href="https://finance.yahoo.com/quote/d/" id="link2" target="_blank">
    D
   </a>
   ,
   <a class="stock" href="https://finance.yahoo.com/quote/exc/" id="link3" target="_blank">
    EXC
   </a>
   ,
   <a class="etf" href="https://finance.yahoo.com/quote/nee/" id="link4" target="_blank">
    NEE
   </a>
   ,
   <a class="stock" href="https://finance.yahoo.com/quote/so/" id="link5" target="_blank">
    SO
   </a>
   ,
were all making headlines.
  </p>
  <p class="details">
   End of HTML document.
  </p>
 </body>
</html>


In [5]:
# Seleccionemos primeros los tags 'a' ten soup (de forma predeterminada, se selecciona la primera aparición de una etiqueta)
tag = soup.a

# Imprimir el tag
print(tag)

<a class="stock" href="https://finance.yahoo.com/quote/duk/" id="link1" target="_blank">DUK</a>


In [6]:
# Mostrar todos los atributos del tag
print(tag.attrs)

{'href': 'https://finance.yahoo.com/quote/duk/', 'target': '_blank', 'class': ['stock'], 'id': 'link1'}


In [7]:
# Acceda al atributo de hipervínculo (use una etiqueta como un diccionario para acceder) 
print(tag['href'])

https://finance.yahoo.com/quote/duk/


In [8]:
# Ver todos los tags de hyperlink en el custom_html_doc
soup.find_all('a')

[<a class="stock" href="https://finance.yahoo.com/quote/duk/" id="link1" target="_blank">DUK</a>,
 <a class="stock" href="https://finance.yahoo.com/quote/d/" id="link2" target="_blank">D</a>,
 <a class="stock" href="https://finance.yahoo.com/quote/exc/" id="link3" target="_blank">EXC</a>,
 <a class="etf" href="https://finance.yahoo.com/quote/nee/" id="link4" target="_blank">NEE</a>,
 <a class="stock" href="https://finance.yahoo.com/quote/so/" id="link5" target="_blank">SO</a>]

In [9]:
for tag in soup.find_all('a'):
    print(tag['href'])

https://finance.yahoo.com/quote/duk/
https://finance.yahoo.com/quote/d/
https://finance.yahoo.com/quote/exc/
https://finance.yahoo.com/quote/nee/
https://finance.yahoo.com/quote/so/


In [10]:
custom_html_doc

'\n<html>\n<head>\n<title>HTML Page Title</title>\n</head>\n<h1>Head: Important Header: Global News</h1>\n<br>\n<h2>Head: Less Imporant Header: Global News</h2>\n<body>\n<p class="title"><b>Paragraph: Financial news</b></p>\n<p class="story"> Stocks had a volatile week, where\n<a href="https://finance.yahoo.com/quote/duk/" target="_blank" class="stock" id="link1">DUK</a>,\n<a href="https://finance.yahoo.com/quote/d/" target="_blank" class="stock" id="link2">D</a>,\n<a href="https://finance.yahoo.com/quote/exc/" target="_blank" class="stock" id="link3">EXC</a>,\n<a href="https://finance.yahoo.com/quote/nee/" target="_blank" class="etf" id="link4">NEE</a>,\n<a href="https://finance.yahoo.com/quote/so/" target="_blank" class="stock" id="link5">SO</a>,\nwere all making headlines.</p>\n<p class="details">End of HTML document.</p>\n'

In [11]:
s = BeautifulSoup(custom_html_doc, 'html.parser')
a_tag_list = s.find_all('a')
a_tag_list

[<a class="stock" href="https://finance.yahoo.com/quote/duk/" id="link1" target="_blank">DUK</a>,
 <a class="stock" href="https://finance.yahoo.com/quote/d/" id="link2" target="_blank">D</a>,
 <a class="stock" href="https://finance.yahoo.com/quote/exc/" id="link3" target="_blank">EXC</a>,
 <a class="etf" href="https://finance.yahoo.com/quote/nee/" id="link4" target="_blank">NEE</a>,
 <a class="stock" href="https://finance.yahoo.com/quote/so/" id="link5" target="_blank">SO</a>]

In [12]:
# Loop en cada uno de los tags
for tag in a_tag_list:
    symbol_name = tag.text
    class_name = tag['class'][0]
    #print(class_name)
    href_name = tag['href']
    print(symbol_name + ',' + class_name + ',' + href_name)

DUK,stock,https://finance.yahoo.com/quote/duk/
D,stock,https://finance.yahoo.com/quote/d/
EXC,stock,https://finance.yahoo.com/quote/exc/
NEE,etf,https://finance.yahoo.com/quote/nee/
SO,stock,https://finance.yahoo.com/quote/so/


In [22]:
import IPython
IPython.display.HTML
file_name="DUK_Yahoo.html"


In [25]:
# IFrame will allow us to view the HTML document
IFrame(src='DUK_Yahoo.html', width=800, height=400)

In [35]:
# Abrir el archivo file y pase el identificador del archivo (aquí el identificador del archivo es f) a BeautifulSoup
file_name = 'file:///C:/Users/Usuario/Documents/Python%20Scripts/DUK_Yahoo.html'
with open(file_name) as f:
  stock_soup = BeautifulSoup(f, 'html.parser')

FileNotFoundError: ignored

In [29]:
# Extraer los primeros 1000 characters y mirar un head del documento (no quiero imprimir demasiado para no quedar desordenado)
print(stock_soup.prettify()[:5000])

NameError: ignored

In [32]:
stock_soup.find_all('td')

NameError: ignored

In [33]:
tag_list = []
for tag in stock_soup.find_all():
    tag_list.append(tag.name)

tag_count_dict = {}
for tag_name in tag_list:
    if tag_name in tag_count_dict:
        tag_count_dict[tag_name] = tag_count_dict[tag_name] + 1
    else:
        tag_count_dict[tag_name] = 1
        
tag_count_dict

NameError: ignored

In [34]:
sorted(tag_count_dict.items(),key=lambda x: x[1],reverse=True)

NameError: ignored

In [37]:
# Nos gustaría seleccionar específicamente la etiqueta que contiene la información de capitalización de mercado para DUK
stock_soup.find_all("td")

NameError: ignored

In [38]:
# Entonces, tneemos que buscar necesitamos buscar por más de una etiqueta -> agregar otro identificador para buscar
stock_soup.find("td", {"data-reactid":"81"}).text

NameError: ignored

In [39]:
for i in stock_soup.find_all("td",{'class':['Ta(end)', 'Fw(b)', 'Lh(14px)']}):
    print(i.attrs['data-test'])

NameError: ignored

In [40]:
# Cargar el archivo
file_name = 'DUK_Yahoo.html'
with open(file_name) as f:
    stock_soup = BeautifulSoup(f, 'html.parser')

# Mi codigo:
bid = stock_soup.find("td", {"data-test" : 'BID-value'}).text
ask = stock_soup.find("td", {"data-test" : 'ASK-value'}).text
volume = stock_soup.find("td", {"data-test" : 'TD_VOLUME-value'}).text
average_volume = stock_soup.find("td", {"data-test" : 'AVERAGE_VOLUME_3MONTH-value'}).text

print("Bid: ", bid)
print("Ask: ", ask)
print("Volume: ", volume)
print("Avg. Volume: ", average_volume)

FileNotFoundError: ignored

In [41]:
# Definir la lista de simbolos a buscar a los que queremos aplicarles el parse
symbol_list = ['NEE','DUK','D','SO','EXC'] 

In [42]:
def process_yahoo(symbol):
    # Cargar los archivos previamente guardados
    file_name = symbol + '_Yahoo.html'
    with open(file_name) as f:
        s = BeautifulSoup(f, 'html.parser')
    
    # Analizar los datos de stock específicos de interés y almacenarlos en un objeto de diccionario 
    info_dict = {'MARKET_CAP' : s.find("td", {"data-test" : 'MARKET_CAP-value'}).text}
    
    return info_dict

# Recorra todos los símbolos, aplicando la función de análisis a cada uno de los archivos HTML correspondientes del símbolo.
fundamental_dict = {}
for sym in symbol_list:
    fundamental_dict[sym] = process_yahoo(sym)

FileNotFoundError: ignored

In [43]:
# Look at the result
fundamental_dict

{}

In [44]:
def process_yahoo(symbol):
    # Definir las variables de interes del mercado a buscar 
    MARKET_CAP= "MARKET_CAP"
    PE_RATIO = "PE_RATIO"
    EPS_RATIO = "EPS_RATIO"
    
    # Cargar previamente los archivos guardados 
    file_name = symbol + '_Yahoo.html'
    with open(file_name) as f:
        s = BeautifulSoup(f, 'html.parser')
    
    # Analizar (Parse) los datos de stock específicos de interés y almacenarlos en un objeto de diccionario
    info_dict = {MARKET_CAP : s.find("td", {"data-test" : MARKET_CAP+'-value'}).text,
                 PE_RATIO : s.find("td", {"data-test" : PE_RATIO+'-value'}).text,
                 EPS_RATIO : s.find("td", {"data-test" : EPS_RATIO+'-value'}).text
                }
    
    return info_dict

# Recorrer todos los símbolos, aplicando la función de análisis a cada uno de los archivos HTML correspondientes del símbolo.
fundamental_dict = {}
for sym in symbol_list:
    fundamental_dict[sym] = process_yahoo(sym)
    
fundamental_dict

FileNotFoundError: ignored

In [45]:
def process_yahoo(symbol):
    # Definir las variables de interes del mercado a buscar 
    MARKET_CAP= "MARKET_CAP"
    PE_RATIO = "PE_RATIO"
    EPS_RATIO = "EPS_RATIO"
    
    # Cargar previamente los archivos guardados 
    file_name = symbol + '_Yahoo.html'
    with open(file_name) as f:
        s = BeautifulSoup(f, 'html.parser')
    
    # Analizar (Parse) los datos de stock específicos de interés y almacenarlos en un objeto de diccionario
    info_dict = {MARKET_CAP : s.find("td", {"data-test" : MARKET_CAP+'-value'}).text,
                 PE_RATIO : s.find("td", {"data-test" : PE_RATIO+'-value'}).text,
                 EPS_RATIO : s.find("td", {"data-test" : EPS_RATIO+'-value'}).text
                }
    
    return info_dict

# Recorrer todos los símbolos, aplicando la función de análisis a cada uno de los archivos HTML correspondientes del símbolo.
fundamental_dict = {}
for sym in symbol_list:
    fundamental_dict[sym] = process_yahoo(sym)
    
fundamental_dict

FileNotFoundError: ignored

### **Live web scraping de data fundamental de stocks**

In [46]:
from io import BytesIO
import gzip
site_url='https://finance.yahoo.com/quote/DUK?p=DUK'
r = urllib.request.urlopen(site_url)
site_content = r.read()
site_content

b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\x00\xec\xbd\xfbz\xdbF\xb28\xf8\xff<\x05\xcc\xc46yL@\xb8\x13$M{dI\x8e=#Y>\x92\x1co\xc6\xc7?~ \x00\x8a\x88A\x80\x01@]\xach\xbf}\x8d}\xbd}\x92\xad\xaan\x80\xb8Q\xa2\xe4\xcb$s\x12;&\xd0\xe8kUu]\xba\xbb\xaa\x9f>\xd8=\xdc9\xf9\xe5\xed\x9e0K\xe7\xc1\xb3\xa7\xf8\xaf\xe0\xbb\xa3\x96\x9dFs\xdfi\tN`\'\xc9\xa8\xf5&\xfaG"8\xb38\x9a{\x82\xeb%\x9f\xd2h!Lm?H\xec\xa9\xd7\x12\x02;<\x1d\xb5\xbcP|w\xdc\x82:<\xdb\x15\x16\xb17\xf5/F\xad\xe8t\x00U\xa7\x8b\xc1\xd6Vt\xba\x90\xe6\xdeV\x98\xfc\x00\x99\x12\'\xf6\x17\xe9\xb3s?t\xa3si\xe1\xc5\xd3(\x9e\xdb\xa1\xe3\t\x8f\x1e\t\xf5Tin\xc7\x9fn\xf8\xd4~\xfc\xd6>\xf5\x8eS;N\x1fw\x86O\xb7x\xf5O\xe7^jC\xbf\xed8\xf1\xd2Qk\x99NE\xab%l={\x9a\xfai\xe0=\xdb]~\xf2\x84\xbd\xd0\x8bO/\x85\x9d(^D\xb1\x9d\xfaQ(\xb4_E\x81\xeb\xc3\xef\xee\xbb\x7fv\x84\xe34r>\toc\xdf\xf1\xba\xc2\x1b\xef<\xe9\n\xff\xbd\x8cR\xe8\xa9=_\x0c\x85W~\x92F\xf1\xa5 \n\xbf\xd8\xb3(\x12^\xfa!v\xeb\xe9\x16k\x83u!\xb4\xe7\xde\xa8\xf5\xc9\xbb<\x8fb7\x01\xb0Fa\xea\x85\xd0%h\xa1+\

In [47]:
type(site_content)

bytes

In [48]:
from io import BytesIO
import gzip

buff = BytesIO(site_content)
f = gzip.GzipFile(fileobj=buff)
site_content = f.read().decode('utf-8')
site_content

'<!DOCTYPE html><html id="atomic" class="NoJs chrome desktop failsafe" lang="en-US"><head prefix="og: http://ogp.me/ns#"><script>window.performance && window.performance.mark && window.performance.mark(\'PageStart\');</script><meta charset="utf-8" /><title>Duke Energy Corporation (Holdin (DUK) Stock Price, News, Quote &amp; History - Yahoo Finance</title><meta name="keywords" content="DUK, Duke Energy Corporation (Holdin, DUK stock chart, Duke Energy Corporation (Holdin stock chart, stock chart, stocks, quotes, finance" /><meta http-equiv="x-dns-prefetch-control" content="on" /><meta property="twitter:dnt" content="on" /><meta property="fb:app_id" content="458584288257241" /><meta name="theme-color" content="#400090" /><meta name="viewport" content="width=device-width, initial-scale=1" /><meta name="description" lang="en-US" content="Find the latest Duke Energy Corporation (Holdin (DUK) stock quote, history, news and other vital information to help you with your stock trading and inves

In [49]:
# Guardar el scraped HTML en un .html file (para procesamiento posterior)
with open('saved_page.html', 'w',encoding='utf-8') as f:
    f.write(site_content)

# Usar el html.parser para crear el soup
s = BeautifulSoup(site_content, 'html.parser')

In [50]:
# Mirar el objeto soup object con el  metodo prettify()
print(s.prettify()[:500]) # Solo mostrar una porcion del texto ya que es largo 

<!DOCTYPE html>
<html class="NoJs chrome desktop failsafe" id="atomic" lang="en-US">
 <head prefix="og: http://ogp.me/ns#">
  <script>
   window.performance && window.performance.mark && window.performance.mark('PageStart');
  </script>
  <meta charset="utf-8"/>
  <title>
   Duke Energy Corporation (Holdin (DUK) Stock Price, News, Quote &amp; History - Yahoo Finance
  </title>
  <meta content="DUK, Duke Energy Corporation (Holdin, DUK stock chart, Duke Energy Corporation (Holdin stock chart, sto


In [51]:
symbol_list = ['NEE','DUK','D','SO','EXC'] # stocks of interest

def scrape_yahoo(symbol):
    symbol_url='https://finance.yahoo.com/quote/' + symbol
    MARKET_CAP= "MARKET_CAP"
    PE_RATIO = "PE_RATIO"
    EPS_RATIO = "EPS_RATIO"
    
    # Scrape
    r = urllib.request.urlopen(symbol_url)
    c = r.read()
    ############ IMPORTANTE EL BytesIO#################
    buff = BytesIO(c)
    f = gzip.GzipFile(fileobj=buff)
    ################################################3
    c = f.read().decode('utf-8')
    s = BeautifulSoup(c, 'html.parser')
    
    info_dict = {MARKET_CAP : s.find("td", {"data-test" : MARKET_CAP+'-value'}).text,
                 PE_RATIO : s.find("td", {"data-test" : PE_RATIO+'-value'}).text,
                 EPS_RATIO : s.find("td", {"data-test" : EPS_RATIO+'-value'}).text
                }
    
    return info_dict

# Scrape los datos, y almacenarlos en un diccionario
symbol_dict = {}
for symbol in symbol_list:
    print("Scraping Symbol: " + symbol)
    symbol_dict[symbol] = scrape_yahoo(symbol)
    
# Mostrar la data con parsing
fundamental_df = pd.DataFrame.from_dict(symbol_dict, orient='index')
fundamental_df

Scraping Symbol: NEE
Scraping Symbol: DUK
Scraping Symbol: D
Scraping Symbol: SO
Scraping Symbol: EXC


,MARKET_CAP,PE_RATIO,EPS_RATIO
NEE,167.822B,139.30,0.61
DUK,78.711B,26.75,3.83
D,61.776B,27.64,2.76
SO,66.23B,21.20,2.95
EXC,52.966B,31.66,1.71
